Get all cemeteries in Wikidata and check OSM
* [This Notebook](https://github.com/salgo60/Gravstensinventeringen-Wikidata/blob/main/Notebook/OSM%20Wikidata%20cemeteryGermany.ipynb)
* [Video](https://youtu.be/EDkPoomRmJg)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-02-15 12:39:55.839839


In [2]:
import pandas as pd
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
# https://w.wiki/6LdN
queryAllCem = """#title FindAgrave xemetery ID no OSM
SELECT ?item ?itemLabel (REPLACE(STR(?item), ".*Q", "Q") AS ?itemid)  WHERE {
  #?item wdt:P6104 wd:Q115206846.
  ?item wdt:P2025 ?FAQid.
  minus {?item wdt:P10689 ?OSM}
  minus {?item wdt:P402 ?OSMrel}

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,sv". }
}
"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDCem = get_sparql_dataframe(endpoint_url, queryAllCem)    
WDCem.shape

(11953, 3)

In [3]:
WDCem.head()

,item,itemLabel,itemid
0,http://www.wikidata.org/entity/Q80146,Saburtalo Pantheon,Q80146
1,http://www.wikidata.org/entity/Q81785,Zachary Taylor National Cemetery,Q81785
2,http://www.wikidata.org/entity/Q82162,Wolvercote Cemetery,Q82162
3,http://www.wikidata.org/entity/Q93544,Monumental Cemetery of Staglieno,Q93544
4,http://www.wikidata.org/entity/Q99309,Pantheon,Q99309


In [4]:
import urllib3, json
from tqdm import tqdm
http = urllib3.PoolManager()

listWD = []
for WD, row in tqdm(WDCem.iterrows(), total=WDCem.shape[0]):
    #print(row["itemid"] ) 
    url = "https://osm.wikidata.link/tagged/api/item/" + row["itemid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["itemid"] 
#    new_item['coord'] = row["coord"] 
    try:
        r = http.request('GET', url) 
        data = json.loads(r.data.decode('utf-8'))
    except:
        print (r.status, url)
#    print (r.status)
    try:
        #print ("OSM ", data["osm"], "Type: ", type(data["osm"]))
        #print ("ID: ", data["osm"][0]["id"] , "\tType: ", data["osm"][0]["type"])
        osmid = data["osm"][0]["id"] 
        osmType = data["osm"][0]["type"]
        new_item['osmid'] = osmid 
        new_item['type'] = osmType
        listWD.append(new_item)
        #print(listWD)

    except:
        #print ("error")
        pass
print (len(listWD))

100%|██████████| 11953/11953 [29:43<00:00,  6.70it/s] 

4598


In [5]:
listWD

[{'wikidata': 'Q80146', 'osmid': 28042077, 'type': 'way'},
 {'wikidata': 'Q81785', 'osmid': 61189473, 'type': 'way'},
 {'wikidata': 'Q82162', 'osmid': 28288191, 'type': 'way'},
 {'wikidata': 'Q93544', 'osmid': 10045286, 'type': 'way'},
 {'wikidata': 'Q99309', 'osmid': 3374342, 'type': 'relation'},
 {'wikidata': 'Q118967', 'osmid': 422679, 'type': 'relation'},
 {'wikidata': 'Q124369', 'osmid': 4481918, 'type': 'way'},
 {'wikidata': 'Q126453', 'osmid': 214435402, 'type': 'way'},
 {'wikidata': 'Q133423', 'osmid': 115773400, 'type': 'way'},
 {'wikidata': 'Q133513', 'osmid': 3666894, 'type': 'relation'},
 {'wikidata': 'Q148554', 'osmid': 66418787, 'type': 'way'},
 {'wikidata': 'Q155225', 'osmid': 153483298, 'type': 'way'},
 {'wikidata': 'Q156427', 'osmid': 28712150, 'type': 'way'},
 {'wikidata': 'Q167087', 'osmid': 1778085175, 'type': 'node'},
 {'wikidata': 'Q167124', 'osmid': 1854240, 'type': 'relation'},
 {'wikidata': 'Q167231', 'osmid': 9078076, 'type': 'relation'},
 {'wikidata': 'Q16888

In [6]:
OSMtot = pd.DataFrame(listWD,
                  columns=['wikidata','osmid','type'])
OSMtot.shape

(4598, 3)

In [7]:
pd.set_option('max_colwidth', 400)
OSMtot.head(10)

,wikidata,osmid,type
0,Q80146,28042077,way
1,Q81785,61189473,way
2,Q82162,28288191,way
3,Q93544,10045286,way
4,Q99309,3374342,relation
5,Q118967,422679,relation
6,Q124369,4481918,way
7,Q126453,214435402,way
8,Q133423,115773400,way
9,Q133513,3666894,relation


In [8]:
OSMtot.to_csv("WD_OSM_Germany_Cemetary.csv")

* [WD_OSM_Germany_Cemetary.csv](https://github.com/salgo60/Gravstensinventeringen-Wikidata/blob/main/Notebook/WD_OSM_Germany_Cemetary.csv)

In [9]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2023-02-15 13:09:45.199171
Time elapsed (hh:mm:ss.ms) 0:29:49.360165
